<a href="https://colab.research.google.com/github/arnabd64/Langchain-Guides/blob/main/notebooks/Langchain_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain

In this notebook we will delve into the following components:

1. RunnableLambda
2. RunnableSequence
3. RunnableParallel

* __What is a Runnable?__:

A [Runnable](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface) is a specialized function built by langchain to ease the construction of complex chains. To execute a `Runnable`, there are 3 methods:

1. `.invoke()`: Calls the `Runnable` on an input.
2. `.stream()`: Streams the response of the `Runnable`.
3. `.batch()`: Calls the `Runnable` on a list of inputs.

__Note__: Mostly all the components in langchain are a `Runnable` which includes prompts, LLMs, chains, agents etc.

# Install Libraries

In [1]:
! pip install --no-cache-dir --progress-bar off \
    langchain==0.2.10 \
    langchain_community==0.2.10 \
    python-dotenv \
    > install.log

In [2]:
import os
import dotenv

if not dotenv.load_dotenv("./.env"):
    raise FileNotFoundError

# Load the Components

In [3]:
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableBranch, RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.globals import set_debug
from langchain.utils.math import cosine_similarity
import math

# `RunnableLambda`

A [`RunnableLambda`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) is an interface between a regular method or `lambda` function and a `Runnable` or in simple words, `RunnableLambda` converts a python method into a `Runnable`. Please note that __a `RunnableLambda` method should only take a single input__.

In [4]:
addition = RunnableLambda(lambda x: x + 9)
multiply = RunnableLambda(lambda y: y * 3)

In [5]:
print("Result of Addition:", addition.invoke(6))
print("Result of Multiplication:", multiply.invoke(4))

Result of Addition: 15
Result of Multiplication: 12


# `RunnableSequence`

A [`RunnableSequence`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html) is a component that allows __chaining__ of multiple `Runnable` and even other `RunnableSequence` instances. This component can be considered as the _chain_ in LangChain. You can create a `RunnableSequence` using the `pipe (|)` operator.

__Example__: Let's calculate the Area of a Circle given the radius. The mathematical formula is $A = \pi r^2$. We can break this operation into 2 parts:

1. Compute the square of `radius` ($r^2$)
2. Multiply with `pi` ($\pi$)

We will be using both `RunnableSequence` as well as `pipe` operator.

In [6]:
# RunnableSequence
circle_area = RunnableSequence(
    RunnableLambda(lambda r: r * r, name="Radius-Squared"),
    RunnableLambda(lambda y: y * math.pi, name="Multiply Pi"),
    name="Area of Circle"
)

# pipe operator
circle_area_pipe = (
    RunnableLambda(lambda r: r * r, name="Radius-Squared")
    | RunnableLambda(lambda y: y * math.pi, name="Multiply Pi")
)

set_debug(True)
print("Area of the Circle =", circle_area.invoke(6))

[chain/start] [chain:Area of Circle] Entering Chain run with input:
{
  "input": 6
}
[chain/start] [chain:Area of Circle > chain:Radius-Squared] Entering Chain run with input:
{
  "input": 6
}
[chain/end] [chain:Area of Circle > chain:Radius-Squared] s] Exiting Chain run with output:
{
  "output": 36
}
[chain/start] [chain:Area of Circle > chain:Multiply Pi] Entering Chain run with input:
{
  "input": 36
}
[chain/end] [chain:Area of Circle > chain:Multiply Pi] s] Exiting Chain run with output:
{
  "output": 113.09733552923255
}
[chain/end] [chain:Area of Circle] s] Exiting Chain run with output:
{
  "output": 113.09733552923255
}
Area of the Circle = 113.09733552923255


In [7]:
print("Area of the Circle =", circle_area_pipe.invoke(2))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": 2
}
[chain/start] [chain:RunnableSequence > chain:Radius-Squared] Entering Chain run with input:
{
  "input": 2
}
[chain/end] [chain:RunnableSequence > chain:Radius-Squared] s] Exiting Chain run with output:
{
  "output": 4
}
[chain/start] [chain:RunnableSequence > chain:Multiply Pi] Entering Chain run with input:
{
  "input": 4
}
[chain/end] [chain:RunnableSequence > chain:Multiply Pi] s] Exiting Chain run with output:
{
  "output": 12.566370614359172
}
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
{
  "output": 12.566370614359172
}
Area of the Circle = 12.566370614359172


# `RunnableParallel`



A [`RunnableParallel`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html#langchain_core.runnables.base.RunnableParallel) allows you to run multiple `RunnableSequence` in parallel. This is useful when you want to run more than one sub-chains in parallel and also can be used inside of a `RunnableSequence`.

## Example: Pros & Cons Generator

To better understand the Parallel execution of the chain, let's take an example.

Our users to want to know the pros & cons of purchasing a product (e.g: A Car, An Electronic etc) that they like. We will first query the LLM to get a product description and then use `RunnableParallel` to retrieve the Pros and the Cons separately in Parallel and then combine those results into a single cohesive text.

### Product Description Chain

A simple chain that generates a product description from a specified product name. This chain looks like:

```markdown
[product-name] --> [PromptTemplate] --> [Ollama] --> [OutputParser]
```

In [8]:
# description prompt
description_prompt = {
    "template": "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: {product}",
    "input_variables": ["product"]
}

# LLM config
config = {
    "name": "Ollama-Local",
    "base_url": os.getenv("HOST"),
    "model": os.getenv("MODEL"),
    "temperature": float(os.getenv("TEMPERATURE")),
    "timeout": int(os.getenv("TIMEOUT")),
    "keep_alive": 3600
}

# build a chain to gather information
query_chain = (
    PromptTemplate(**description_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

### Pros Chain

This chain will generate the Pros of purchasing the above product using the product description generated by the LLM using the Product Description Chain.

```markdown
[product-description] --> [PromptTemplate] --> [Ollama] --> [OutputParser]
```

In [9]:
# Pros Chain
#Prompt template
pros_prompt = {
    "input_variables": ["description"],
    "template": "Write down exactly 5 points on why should the user purchase a product according to the following product description:\n{description}"
}

# build the chain
pros_chain = (
    PromptTemplate(**pros_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

### Cons Chain

Similar to the Pros chain, the Cons chain is used to generate the cons of purchasing the product using the product description generated by the LLM using the Product Description Chain.

```markdown
[product-description] --> [PromptTemplate] --> [Ollama] --> [OutputParser]
```

In [10]:
# Cons Chain
# Prompt
cons_prompt = {
    "input_variables": ["description"],
    "template": "Write down exactly 5 points on why the user should not purchase a product according to the following product description:\n{description}"
}

# cons chain
cons_chain = (
    PromptTemplate(**cons_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

### Combine Chain

This chain will combine both the outputs of both the Pros Chain and Cons Chain and generate a single pros & cons summary.

```markdown
[Pros-Chain]
    |
    |---->
            [PromptTemplate] ---> [Ollama] ---> [OutputParser]
    |---->
    |
[Cons-Chain]
```

In [11]:
# Combine the Chains
combine_prompt = {
    "input_variables": ["pros", "cons"],
    "template": "You are tasked with summarizing the advantages and disadvantages of purchasing a product from the given list: Firstly, the advatages:\n{pros} and finally the disadvantages:\n{cons}"
}

# combine chain
combine_chain = (
    PromptTemplate(**combine_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

### Chaining all things Together

Now we link all the chains together to form a single chain that takes in the name of a product and generates a single response containing the pros and cons of that product.

In [12]:
final_chain = (
    query_chain
    | RunnableParallel(pros=pros_chain, cons=cons_chain)
    | combine_chain
)

### Run the Chain

Use `set_debug(True)` to print the status of the chain in the notebook.

In [13]:
set_debug(False)
response = final_chain.invoke({"product": "Porsche 911 Turbo GT"})

In [14]:
print(response)

 Advantages of purchasing the Porsche 911 Turbo GT include:
1. High-Performance: Expected 0-60 mph acceleration of around 2.8 seconds and a top speed exceeding 198 mph.
2. Luxury Brand: Renowned for superior craftsmanship, quality, and prestige.
3. Advanced Technology: Equipped with a turbocharged flat-six engine, PDK dual-clutch automatic transmission, and potential active aerodynamics.
4. Aggressive Aerodynamics: Improved downforce and handling for both the track and road.
5. Superior Handling: Upgraded suspension components, larger brakes, and possibly active aerodynamics ensure maximum control and stability.
6. Luxurious Interior: Features Alcantara trim, sports seats, a digital instrument cluster, and infotainment system with Apple CarPlay and Android Auto compatibility.
7. Potential for Customization: Opportunities to customize the car according to personal preferences or performance needs.

Disadvantages include:
1. Unconfirmed Details: Many details such as release date, price, 

## A Simpler way to Code

In the following section, we will code the above example but in a more organized manner. Here we  will be using a python dictionary to contain all the parameters and configurations necessary to run the example.

In [15]:
chain_config = {
    # Prompt Templates
    "prompts": {
        # get the product description
        "description": {
            "name": "Prompt-Generate-Product_Description",
            "input_variables": ["product"],
            "template": "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: {product}"
        },

        # get the Pros
        "pros": {
            "name": "Prompt-Generate-Advantages",
            "input_variables": ["description"],
            "template": "List exactly 5 reasons why the user should purchase the product described as:\n{description}"
        },

        # get the Cons
        "cons": {
            "name": "Prompt-Generate-Disadvantages",
            "input_variables": ["description"],
            "template": "List exactly 5 reasons why the user should not purchase the product described as:\n{description}"
        },

        # combine the pros and cons
        "combine": {
            "name": "Prompt-Summarize-Outputs",
            "input_variables": ["pros", "cons"],
            "template": "Write down 3 points each on the Advantages and Disadvantages of making a purchase for a product. The advantages of a purcahse are:\n{pros} and the disadvantages of a purcashe are:\n{cons}"
        }
    },

    # Configuration for the LLM
    "llm-config": {
        "name": "Ollama-Local",
        "base_url": os.getenv("HOST"),
        "model": os.getenv("MODEL"),
        "temperature": float(os.getenv("TEMPERATURE")),
        "timeout": int(os.getenv("TIMEOUT")),
        "keep_alive": 3600
    }
}

### Using the `pipe` (`|`) Operator to Build the Chain

In [16]:
chain_pipe = (
    # 1. Description Generator Chain
    PromptTemplate(**chain_config["prompts"]["description"])
    | Ollama(**chain_config["llm-config"])
    | StrOutputParser()

    # 2. Parallel chains
    | RunnableParallel(
        # 2A. Generate Advantages
        pros = (
            PromptTemplate(**chain_config["prompts"]["pros"])
            | Ollama(**chain_config["llm-config"])
            | StrOutputParser()
        ),
        # 2B. Generate Disadvantages
        cons = (
            PromptTemplate(**chain_config["prompts"]["cons"])
            | Ollama(**chain_config["llm-config"])
            | StrOutputParser()
        )
    )

    # 3. Combine the Advantages & Disadvantages
    | PromptTemplate(**chain_config["prompts"]["combine"])
    | Ollama(**chain_config["llm-config"])
    | StrOutputParser()
)

In [17]:
set_debug(False)
response = chain_pipe.invoke({"product": "Porsche 911 Carrera RS"})

In [18]:
print(response)

 Advantages of making a purchase:

1. Satisfaction of Ownership: Purchasing a product can provide a sense of accomplishment and satisfaction, especially if the product is something desired for a long time. It allows the owner to have personal possession and control over the product.

2. Usefulness and Convenience: A purchased product can bring practical benefits such as solving a problem, fulfilling a need, or enhancing daily life in some way. For example, a new phone may offer improved features for communication and entertainment.

3. Quality and Durability: Many products, especially those of high quality, are built to last and provide reliable service over an extended period. This can result in long-term savings compared to repeatedly buying lower-quality alternatives.

Disadvantages of making a purchase:

1. High Costs: Purchasing a product often comes with a financial commitment that may be significant, especially for higher-end or luxury items. The cost of the product, as well as 

### Using `RunnableSequence` to Build the Chain

In [19]:
chain_subchain = RunnableSequence(
    # 1. Product Description Generator Sub-Chain
    RunnableSequence(
        PromptTemplate(**chain_config["prompts"]["description"]),
        Ollama(**chain_config["llm-config"]),
        StrOutputParser(),
        name="Chain-Generate-Product_Description"
    ),

    # 2. Generate Advantages & Disadvantages
    RunnableParallel(
        pros = RunnableSequence(
            # 2A. Generate the Advantages
            PromptTemplate(**chain_config["prompts"]["pros"]),
            Ollama(**chain_config["llm-config"]),
            StrOutputParser(),
            name="Chain-Generate-Advantages"
        ),
        cons = RunnableSequence(
            # 2B. Generate the Disadvantages
            PromptTemplate(**chain_config["prompts"]["cons"]),
            Ollama(**chain_config["llm-config"]),
            StrOutputParser(),
            name="Chain-Generate-Disadvantages"
        )
    ),

    # 3. Combine the Results
    RunnableSequence(
        PromptTemplate(**chain_config["prompts"]["combine"]),
        Ollama(**chain_config["llm-config"]),
        StrOutputParser(),
        name="Chain-Summarize-Output"
    ),
    name="Parent-Chain"
)

In [20]:
set_debug(False)
chain_subchain.invoke({"product": "Porsche Carrera RS"})

" Thank you for your detailed response! Here's a brief summarization of the advantages and disadvantages of purchasing a product like the Porsche Carrera RS:\n\nAdvantages:\n1. Design: Iconic, timeless design that is both aesthetically pleasing and distinctive.\n2. Engine: High-revving flat-six air-cooled engine delivers impressive power and performance.\n3. Performance: Exceptional handling and high speed capabilities make for an exhilarating driving experience.\n4. Versions: Various models available to cater to different preferences and requirements.\n5. Legacy: A significant influence on automotive history, the Carrera RS continues to impact modern Porsche models and sports cars in general.\n\nDisadvantages:\n1. Cost: High demand and rarity result in a steep price tag that may be prohibitive for some buyers.\n2. Maintenance: Requires regular maintenance and restoration work, which can be costly and time-consuming.\n3. Availability: Finding an original, well-maintained Carrera RS can